# Plot a Resource Model Entity-relationships diagram

Creates the Entity-relationships diagram (ERD) of EAMENA Heritage Places (HP) and color nodes according to the [mds-template-readonly.tsv](https://github.com/eamena-project/eamena-arches-dev/blob/main/dbs/database.eamena/data/reference_data/mds/mds-template-readonly.tsv) file (see: [GitHub documentation](https://github.com/eamena-project/eamena-arches-dev/tree/main/dbs/database.eamena/data/reference_data#templates))

Load GitHub repository, requirements and libraries

In [1]:
# download ACHP graph_mixer
!rm cultural-heritage -R
!git clone https://github.com/achp-project/cultural-heritage.git
!pip install  -q -r  cultural-heritage/graph-parser/requirements.txt
%cd /content/cultural-heritage/graph-parser/
import graph_mixer as gm

# download EAMENA erms
%cd /content/
!rm eamena-functions -R
!git clone https://github.com/eamena-project/eamena-functions.git
%cd /content/eamena-functions/mds
import mds

# librairies
import os
import urllib.request
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import re
from IPython.display import HTML
from pyvis import network as net
import folium
import json
from google.colab import files

Cloning into 'cultural-heritage'...
remote: Enumerating objects: 3206, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (157/157), done.
remote: Total 3206 (delta 310), reused 440 (delta 290), pack-reused 2732
Receiving objects: 100% (3206/3206), 36.14 MiB | 16.97 MiB/s, done.
Resolving deltas: 100% (2344/2344), done.
/content/cultural-heritage/graph-parser
/content
Cloning into 'eamena-functions'...
remote: Enumerating objects: 275, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 275 (delta 33), reused 52 (delta 21), pack-reused 210
Receiving objects: 100% (275/275), 59.00 KiB | 1.97 MiB/s, done.
Resolving deltas: 100% (156/156), done.
/content/eamena-functions/mds


## Select

Select the EAMENA HP Resource Model (RM) within the listed RMs

List of available RMs

In [2]:
%cd /content/cultural-heritage/graph-parser/
!rm inputResourceModels -R
!mkdir -p inputResourceModels
remote_source_files = gm.rm_list()
remote_source_files

/content/cultural-heritage/graph-parser
rm: cannot remove 'inputResourceModels': No such file or directory


{'CAAL': 'https://raw.githubusercontent.com/achp-project/prj-caal/main/resource_models/arches/CAAL-SitesAndMonuments.json',
 'EAMENA': 'https://raw.githubusercontent.com/achp-project/prj-eamena-marea/main/resource_models/Heritage%20Place.json',
 'MAEASAM': 'https://raw.githubusercontent.com/achp-project/prj-maeasam/main/Site.json',
 'MAHS': 'https://raw.githubusercontent.com/achp-project/prj-mahs/main/Site.json',
 'MAHSA': 'https://raw.githubusercontent.com/achp-project/prj-mahsa/main/resource-models/Heritage%20Location%20Resource%20Model.json',
 'MAPHSA': 'https://raw.githubusercontent.com/achp-project/prj-maphsa/heritage_branch/resource_model/MAPHSA%20Heritage%20Location%20Resource%20Model%20v0.44.json',
 'MAPSS': 'https://raw.githubusercontent.com/achp-project/prj-mapss/main/pkg/graphs/Heritage%20Place%20(3).json'}

Select EAMENA RM (key: `EAMENA`) to plot it (nb, other keys: `MAPSS`, `MAHS`, etc.)

In [3]:
gm.rm_selected_one('EAMENA', remote_source_files)
# TODO: return a message showin that the subgraphMetrics.tsv has been created

## Read

Create subgraph and comparison dataframe csv files by running [graph-parser.py](https://github.com/achp-project/cultural-heritage/blob/main/graph-parser/graph_parser.py) and [graph-comparator.py](https://github.com/achp-project/cultural-heritage/blob/main/graph-parser/graph_comparator.py)

In [ ]:
!python /content/cultural-heritage/graph-parser/graph_comparator.py -d -m list -o subgraphMetrics.csv inputResourceModels/*.json
subgraph_metrics = pd.read_csv('subgraphMetrics.csv')
subgraph_metrics

## Show

In [6]:
rm_project= 'EAMENA'
filename = rm_project + ".html"
mds_template_df = mds.mds_template()
mds_template_df['Enhanced record minimum standard'] = mds_template_df['Enhanced record minimum standard'].str.contains(r'Yes', case = False, na = False, regex = True).astype(int)
mds_template_df = mds_template_df.loc[mds_template_df['Enhanced record minimum standard'] == 1]
in_erms = list(mds_template_df['uuid_sql'])
## previously with mds.mds_field_colors(), see ACHP ---------------------
# df_color = mds.mds_field_colors()
# G = gm.create_rm_graph(rm_project= rm_project, color_fields = df_color)
## ----------------------------------------------------------------------
G = gm.create_rm_graph(rm_project= rm_project, color_fields = mds_template_df)
hp_G = gm.plot_net_graph(G, filename = filename, width = "1200px", height = "1000px")
HTML(filename = filename)
## MDS fields should be colored according to the MDS colors (mds-readonly.tsv) but some of them are missing
## probably an error on their UUID in mds-template.xlsx, see: https://github.com/eamena-project/eamena-arches-dev/tree/main/dbs/database.eamena/data/reference_data#templates
## Missing colored fields:
## - Effect Type
## - Disturbance Date To
## - Threat Cause Category Type
## - (not the appropriate color) Location Certainty (also the other geometry field)


## Export

In [ ]:
files.download(filename)